## Project Stage - III (Basic Machine Learning)

## Goals

The goal of Stage II is to utlize machine learning and statistical models to predict the trend of COVID-19 cases / deaths.

### Tasks:

#### Task 1: (70 pts)
- Team: (30)
    - Develop Linear and Non-Linear (polynomial) regression models for predicting cases and deaths in US. 
        - Start your data from the first day of infections in US. X-Axis - number of days since the first case, Y-Axis - number of new cases and deaths.
        - Calculate and report Root Mean Square Error (RMSE) for your models (linear and non-linear). Discuss bias versus variance tradeoff.
        - Plot trend line along for the data along with the forecast of 1 week ahead. 
        - Describe the trends as compared to other countries. 
- Member: (40 pts)
    - Utilize Linear and Non-Linear (polynomial) regression models to compare trends for a single state and its counties (top 5 with highest number of cases). Start your data from the first day of infections. 
        - X-Axis - number of days since the first case, Y - Axis number of new cases and deaths. Calcluate error using RMSE.
        - Identify which counties are most at risk. Model for top 5 counties with cases within a state and describe their trends.
        - Utilize the hospital data to calculate the point of no return for a state. Use percentage occupancy / utilization to see which states are close and what their trend looks like.
     - Perform hypothesis tests on questions identified in Stage II
        - e.x. *Does higher employment data (overall employment numbers) lead to higher covid case numbers or more rapid increase in covid cases.*. Here you would compare the covid cases to the state or county level enrichment data to prove or disprove your null hypothesis. In this case there will be a two tail - two sample t-test to see if there is a difference and then one-tail - two sample t-test to show higher or lower.
        - Depending on your type of data you can also perform Chi-square test for categorical hypothesis testing. 

    
#### Task 2: (30 pts)
- Member:
    - For each of the aforemention analysis plot graphs,
        - trend line
        - confidence intervals (error in prediction)
        - prediction path (forecast)

**Deliverable**
- Each member creates separate notebooks for member tasks. Upload all notebooks and reports to Github Repository. 
- Presentation recordings on canvas.

## Deadline: 11/18/2021

## Linear regression model

In [5]:
import pandas as pd

In [6]:
df_all = pd.read_csv("D:/Kiptaror/CSC-405-605_Fall_2021-master/data/owid-covid-data.csv")

In [7]:
df_all

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133494,ZWE,Africa,Zimbabwe,2021-11-11,133329.0,27.0,31.000,4694.0,0.0,1.286,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
133495,ZWE,Africa,Zimbabwe,2021-11-12,133329.0,0.0,31.000,4694.0,0.0,1.286,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
133496,ZWE,Africa,Zimbabwe,2021-11-13,133393.0,64.0,32.143,4696.0,2.0,1.571,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
133497,ZWE,Africa,Zimbabwe,2021-11-14,133428.0,35.0,34.429,4696.0,0.0,1.571,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN


In [1]:
def country_calculation(df_all,state,date,day):
    df_country = df_all.copy()
    df_country = df_country.loc[df_country['date'] >= date]
    df_country = df_country.loc[df_country['state'] == country_dict[country]]
    features = ['iso_code', 'continent', 'location','total_cases', 'total_deaths', 'date']
    df_country = df_country[features]
    
    # Lags
    df_country = lag_feature(df_country, 'total_cases',range(1, 40))
    df_country = lag_feature(df_country, 'total_deaths', range(1,20))

    filter_col_confirmed = [col for col in df_country if col.startswith('total_cases')]
    filter_col_fatalities= [col for col in df_country if col.startswith('death')]
    filter_col = np.append(filter_col_confirmed, filter_col_fatalities)
    
    # Apply log transformation
    df_country[filter_col] = df_country[filter_col].apply(lambda x: np.log1p(x))
    df_country.replace([np.inf, -np.inf], 0, inplace=True)
    df_country.fillna(0, inplace=True) ####
    
    # Start/end of forecast
    start = df_country[df_country['iso_code']==-999].Day_num.min()
    end = df_country[df_country['iso_code']==-999].Day_num.max()
    #
    for d in range(start,end+1):
   
        X_train_1, X_train_2, Y_train_1, Y_train_2, x_test_1, x_test_2 = train_test_split_extend(df_country,d,day,filter_col_confirmed,filter_col_fatalities)
        
        regr_1, pred_1 = lin_reg(X_train_1, Y_train_1, x_test_1)
        df_country.loc[(df_country['Day_num'] == d) & (df_country['state'] == country_dict[country]), 'total_cases'] = pred_1[0]
        
        regr_2, pred_2 = lin_reg(X_train_2, Y_train_2, x_test_2)
        df_country.loc[(df_country['Day_num'] == d) & (df_country['Country'] == country_dict[country]), 'death'] = pred_2[0]
        
        df_country = lag_feature(df_country, 'total_cases',range(1, 40))
        df_country = lag_feature(df_country, 'death', range(1,20))

        df_country.replace([np.inf, -np.inf], 0, inplace=True)
        df_country.fillna(0, inplace=True)
        
    print("Calculation done.")
    return df_country
    
    df_check = country_calculation(df_all, 'Germany', '2020-03-10', 48)

In [8]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133499 entries, 0 to 133498
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    133499 non-null  object 
 1   continent                                   124973 non-null  object 
 2   location                                    133499 non-null  object 
 3   date                                        133499 non-null  object 
 4   total_cases                                 126238 non-null  float64
 5   new_cases                                   126236 non-null  float64
 6   new_cases_smoothed                          125193 non-null  float64
 7   total_deaths                                115089 non-null  float64
 8   new_deaths                                  115285 non-null  float64
 9   new_deaths_smoothed                         125193 non-null  float64
 

In [1]:
def country_calculation(df_all,state,date,day):
    df_country = df_all.copy()
    df_country = df_country.loc[df_country['date'] >= date]
    df_country = df_country.loc[df_country['state'] == country_dict[country]]
    features = ['Id', 'State', 'location','ConfirmedCases', 'deaths', 'Day_num']
    df_country = df_country[features]
    
    # Lags
    df_country = lag_feature(df_country, 'ConfirmedCases',range(1, 40))
    df_country = lag_feature(df_country, 'deaths', range(1,20))

    filter_col_confirmed = [col for col in df_country if col.startswith('Confirmed')]
    filter_col_fatalities= [col for col in df_country if col.startswith('deaths')]
    filter_col = np.append(filter_col_confirmed, filter_col_fatalities)
    
    # Apply log transformation
    df_country[filter_col] = df_country[filter_col].apply(lambda x: np.log1p(x))
    df_country.replace([np.inf, -np.inf], 0, inplace=True)
    df_country.fillna(0, inplace=True) ####
    
    # Start/end of forecast
    start = df_country[df_country['iso_code']==-999].Day_num.min()
    end = df_country[df_country['iso_code']==-999].Day_num.max()
    #
    for d in range(start,end+1):
   
        X_train_1, X_train_2, Y_train_1, Y_train_2, x_test_1, x_test_2 = train_test_split_extend(df_country,d,day,filter_col_confirmed,filter_col_fatalities)
        
        regr_1, pred_1 = lin_reg(X_train_1, Y_train_1, x_test_1)
        df_country.loc[(df_country['Day_num'] == d) & (df_country['state'] == country_dict[country]), 'ConfirmedCases'] = pred_1[0]
        
        regr_2, pred_2 = lin_reg(X_train_2, Y_train_2, x_test_2)
        df_country.loc[(df_country['Day_num'] == d) & (df_country['state'] == country_dict[country]), 'deaths'] = pred_2[0]
        
        df_country = lag_feature(df_country, 'ConfirmedCases',range(1, 40))
        df_country = lag_feature(df_country, 'deaths', range(1,20))

        df_country.replace([np.inf, -np.inf], 0, inplace=True)
        df_country.fillna(0, inplace=True)
        
    print("Calculation done.")
    return df_country
    
    df_check = country_calculation(df_all, 'Germany', '2020-03-10', 48)

In [ ]:
# Defining key dates for reference purposes #
feature_day = [1,20,50,100,200,500,1000]
def CreateInput(data):
    feature = []
    for day in feature_day:
        data.loc[:,'Number day from ' + str(day) + ' case'] = 0
        if (train[(train['continent'] == country) & (train['location'] == province) & (train['ConfirmedCases'] < day)]['date'].count() > 0):
            fromday = train[(train['continent'] == country) & (train['location'] == province) & (train['ConfirmedCases'] < day)]['date'].max()        
        else:
            fromday = train[(train['continent'] == country) & (train['location'] == province)]['date'].min()       
        for i in range(0, len(data)):
            if (data['date'].iloc[i] > fromday):
                day_denta = data['date'].iloc[i] - fromday
                data['Number day from ' + str(day) + ' case'].iloc[i] = day_denta.days 
        feature = feature + ['Number day from ' + str(day) + ' case']
    
    return data[feature]
    
pred_data_all = pd.DataFrame()
for country in train['continent'].unique():
    for province in train[(train['continent'] == country)]['location'].unique():
        print(country + ' and ' + province)
        #create dataframe for a specific country
        df_train = train[(train['continent'] == country) & (train['location'] == province)]
        df_test = test[(test['continent'] == country) & (test['location'] == province)]
        #create features -> number of cases on a specific date
        X_train = CreateInput(df_train)
        #last 12 confirmed cases in train data set
        y_train_confirmed = df_train['ConfirmedCases'].ravel()
        #last 12 confirmed fatalities in train data set
        y_train_fatalities = df_train['death'].ravel()
        #create features in test dataset-> number of cases on a specific date
        X_pred = CreateInput(df_test)
        #creates reversed list of the possible features
        for day in sorted(feature_day,reverse = True):
            #check for the column in the list
            feature_use = 'Number day from ' + str(day) + ' case'
            #check the 0-dimension of the array (similiar to length of a dataframe)
            idx = X_train[X_train[feature_use] == 0].shape[0]     
            #if there are more than 20 values for a column, the loop will be interruped
            if (X_train[X_train[feature_use] > 0].shape[0] >= 20):
                break
                
        #[TRAIN] - cuts the value of idx from the top of the dataframe; selects the input column (e.g. Number day from 1000 case); brings it into a horizontal array
        adjusted_X_train = X_train[idx:][feature_use].values.reshape(-1, 1)
        #[TRAIN] - get the respective confirmed cases
        adjusted_y_train_confirmed = y_train_confirmed[idx:]
        #[TRAIN] - get the respective fatalities
        adjusted_y_train_fatalities = y_train_fatalities[idx:] #.values.reshape(-1, 1)
        
        #[TEST] - selects for the extracted feature column and get the length (0)
        idx = X_pred[X_pred[feature_use] == 0].shape[0]
        
        #[TEST] - creates a clean array also for the prediction
        adjusted_X_pred = X_pred[idx:][feature_use].values.reshape(-1, 1)
        
        #[TEST] - gets the extract from the test dataset for a specific country/ region
        pred_data = test[(test['continent'] == country) & (test['location'] == province)]
        
        #latest date from the trainings data set and earliest date from the test data set
        max_train_date = train[(train['continent'] == country) & (train['location'] == province)]['Date'].max()
        min_test_date = pred_data['date'].min()
        
        if len(adjusted_y_train_confirmed) < 1:
            adjusted_y_train_confirmed = np.zeros(3)
        else:
            if len(adjusted_y_train_confirmed) < 2:
                adjusted_y_train_confirmed = np.append(adjusted_y_train_confirmed,adjusted_y_train_confirmed[len(adjusted_y_train_confirmed)-1],adjusted_y_train_confirmed[len(adjusted_y_train_confirmed)-1])
            else:
                if len(adjusted_y_train_confirmed) < 3:
                    adjusted_y_train_confirmed = np.append(adjusted_y_train_confirmed,adjusted_y_train_confirmed[len(adjusted_y_train_confirmed)-1])
                else:
                    pass
        
        #[CONFIRMED CASES] - prediction and modelling
        model = SARIMAX(adjusted_y_train_confirmed, order=(1,1,0), 
                        measurement_error=True).fit(disp=False)
        y_hat_confirmed = model.forecast(pred_data[pred_data['Date'] > max_train_date].shape[0])
        y_train_confirmed = train[(train['continent'] == country) & (train['location'] == province) & (train['date'] >=  min_test_date)]['ConfirmedCases'].values
        y_hat_confirmed = np.concatenate((y_train_confirmed,y_hat_confirmed), axis = 0)

        if len(adjusted_y_train_fatalities) < 1:
            adjusted_y_train_fatalities = np.zeros(3)
        else:
            if len(adjusted_y_train_fatalities) < 2:
                adjusted_y_train_fatalities = np.append(adjusted_y_train_fatalities,adjusted_y_train_fatalities[len(adjusted_y_train_fatalities)-1],adjusted_y_train_fatalities[len(adjusted_y_train_fatalities)-1])
            else:
                if len(adjusted_y_train_fatalities) < 3:
                    adjusted_y_train_fatalities = np.append(adjusted_y_train_fatalities,adjusted_y_train_fatalities[len(adjusted_y_train_fatalities)-1])
                else:
                    pass
                
        #[FATALITIES] - prediction and modelling
        model = SARIMAX(adjusted_y_train_fatalities, order=(1,1,0), 
                        measurement_error=True).fit(disp=False)
        y_hat_fatalities = model.forecast(pred_data[pred_data['Date'] > max_train_date].shape[0])
        y_train_fatalities = train[(train['continent'] == country) & (train['location'] == province) & (train['date'] >=  min_test_date)]['Fatalities'].values
        y_hat_fatalities = np.concatenate((y_train_fatalities,y_hat_fatalities), axis = 0)
        pred_data['ConfirmedCases_hat'] =  y_hat_confirmed
        pred_data['Fatalities_hat'] = y_hat_fatalities
        pred_data_all = pred_data_all.append(pred_data)